#### The axis labeling information in pandas objects serves many purposes:

- Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display.
- Enables automatic and explicit data alignment.
- Allows intuitive getting and setting of subsets of the data set.

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

> Note The Python and NumPy indexing operators [] and attribute operator . provide quick and easy access to pandas data structures across a wide range of use cases. This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays. However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized pandas data access methods exposed in this chapter.

## Basics

The following table shows return type values when indexing pandas objects with []:

| Object Type | Selection      | Return Value Type               |
|-------------|----------------|---------------------------------|
| Series      | series[label]  | scalar value                    |
| DataFrame   | frame[colname] | Series corresponding to colname |

In [1]:
import pandas as pd
import numpy as np

In [2]:
dates = pd.date_range('1/1/2000', periods=8)

df = pd.DataFrame(np.random.randn(8, 4), index=dates, columns=['A', 'B', 'C', 'D'])
df.head()

,A,B,C,D
2000-01-01,0.971047,0.542045,-0.495519,0.137423
2000-01-02,0.126074,-0.886903,0.279908,-2.781176
2000-01-03,1.287311,-1.199292,1.569447,-0.880195
2000-01-04,-0.784285,-0.024566,0.388444,-0.683443
2000-01-05,-0.776495,-0.320646,-1.371365,-0.302604


Thus, as per above, we have the most basic indexing using []:

In [3]:
s = df['A']
s[dates[5]]

-0.3885082169730956

It is very useful to apply a transform (in-place) to a subset of the columns (cannot be done similarly with loc and iloc):

In [4]:
df.head()

,A,B,C,D
2000-01-01,0.971047,0.542045,-0.495519,0.137423
2000-01-02,0.126074,-0.886903,0.279908,-2.781176
2000-01-03,1.287311,-1.199292,1.569447,-0.880195
2000-01-04,-0.784285,-0.024566,0.388444,-0.683443
2000-01-05,-0.776495,-0.320646,-1.371365,-0.302604


In [5]:
df[['B', 'A']] = df[['A', 'B']]
df.head()

,A,B,C,D
2000-01-01,0.542045,0.971047,-0.495519,0.137423
2000-01-02,-0.886903,0.126074,0.279908,-2.781176
2000-01-03,-1.199292,1.287311,1.569447,-0.880195
2000-01-04,-0.024566,-0.784285,0.388444,-0.683443
2000-01-05,-0.320646,-0.776495,-1.371365,-0.302604


## Attribute access

You may access an index on a Series or column on a DataFrame directly as an attribute:

In [6]:
df.A

2000-01-01    0.542045
2000-01-02   -0.886903
2000-01-03   -1.199292
2000-01-04   -0.024566
2000-01-05   -0.320646
2000-01-06   -0.665214
2000-01-07    0.345925
2000-01-08   -0.286089
Freq: D, Name: A, dtype: float64

In [7]:
sa = pd.Series([1, 2, 3], index=list('abc'))
sa.b

2

## Slicing

With DataFrame, slicing inside of [] slices the rows. This is provided largely as a convenience since it is such a common operation.

In [8]:
df[::-1]

,A,B,C,D
2000-01-08,-0.286089,0.294147,0.467222,-0.796261
2000-01-07,0.345925,-0.122817,0.797087,-3.073061
2000-01-06,-0.665214,-0.388508,-0.034068,1.827295
2000-01-05,-0.320646,-0.776495,-1.371365,-0.302604
2000-01-04,-0.024566,-0.784285,0.388444,-0.683443
2000-01-03,-1.199292,1.287311,1.569447,-0.880195
2000-01-02,-0.886903,0.126074,0.279908,-2.781176
2000-01-01,0.542045,0.971047,-0.495519,0.137423


## Selection by label

pandas provides a suite of methods in order to have purely label based indexing. This is a strict inclusion based protocol. Every label asked for must be in the index, or a KeyError will be raised. When slicing, both the start bound AND the stop bound are included, if present in the index. Integers are valid labels, but they refer to the label and not the position.

The .loc attribute is the primary access method. The following are valid inputs:

- A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

- A list or array of labels ['a', 'b', 'c'].
- A slice object with labels 'a':'f' (Note that contrary to usual python slices, both the start and the stop are included, when present in the index! See Slicing with labels.
- A boolean array.
- A callable, see Selection By Callable.

In [9]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))

s1.loc['c':]

c    0.935358
d   -0.443303
e    2.078960
f    1.075126
dtype: float64

In [10]:
s1.loc['b']

0.17712964058191943

Note that setting works as well:

In [11]:
s1.loc['c':] = 0
s1

a   -2.669998
b    0.177130
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

With a DataFrame:

In [12]:
df1 = pd.DataFrame(np.random.randn(6, 4), 
                   index=list('abcdef'),
                   columns=list('ABCD'))
df1.head()

,A,B,C,D
a,0.353738,0.127227,-0.661476,0.717148
b,-0.160520,-0.322845,1.517146,-0.148533
c,2.558492,1.966952,0.464750,0.583828
d,1.738518,-0.757292,-0.859018,1.276810
e,-0.128382,0.959006,-2.361875,-0.756041


In [13]:
df1.loc[['a', 'b', 'd'], :]

,A,B,C,D
a,0.353738,0.127227,-0.661476,0.717148
b,-0.160520,-0.322845,1.517146,-0.148533
d,1.738518,-0.757292,-0.859018,1.276810


Accessing via label slices:

In [14]:
df1.loc['d':, 'A':'C']

,A,B,C
d,1.738518,-0.757292,-0.859018
e,-0.128382,0.959006,-2.361875
f,1.506578,1.152356,1.355340


For getting a cross section using a label (equivalent to df.xs('a')):

In [15]:
df1.loc['a']

A    0.353738
B    0.127227
C   -0.661476
D    0.717148
Name: a, dtype: float64

For getting values with a boolean array:

In [16]:
df1.loc['a'] > 0

A     True
B     True
C    False
D     True
Name: a, dtype: bool

In [17]:
df1.loc[:, df1.loc['a'] > 0]

,A,B,D
a,0.353738,0.127227,0.717148
b,-0.160520,-0.322845,-0.148533
c,2.558492,1.966952,0.583828
d,1.738518,-0.757292,1.276810
e,-0.128382,0.959006,-0.756041
f,1.506578,1.152356,1.140441


For getting a value explicitly

In [18]:
df1.loc['a', 'A']

0.3537380098535124

## Slicing with labels

In [19]:
s = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])
s.loc[3:5]

3    b
2    c
5    d
dtype: object

If at least one of the two is absent, but the index is sorted, and can be compared against start and stop labels, then slicing will still work as expected, by selecting labels which rank between the two:

In [20]:
s.sort_index()

0    a
2    c
3    b
4    e
5    d
dtype: object

In [21]:
s.sort_index().loc[1:6]

2    c
3    b
4    e
5    d
dtype: object

## Selection by position

Pandas provides a suite of methods in order to get purely integer based indexing. The semantics follow closely Python and NumPy slicing. These are 0-based indexing. When slicing, the start bound is included, while the upper bound is excluded. Trying to use a non-integer, even a valid label will raise an IndexError.

The .iloc attribute is the primary access method. The following are valid inputs:

- An integer e.g. 5.
- A list or array of integers [4, 3, 0].
- A slice object with ints 1:7.
- A boolean array.
- A callable, see Selection By Callable.

In [22]:
 s1 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))

In [23]:
s1

0    0.647571
2    1.262127
4    0.468873
6    0.612121
8    0.107659
dtype: float64

In [24]:
s1.iloc[:3]

0    0.647571
2    1.262127
4    0.468873
dtype: float64

In [25]:
s1.iloc[3]

0.6121214935755164

In [26]:
s1.iloc[:3] = 0
s1

0    0.000000
2    0.000000
4    0.000000
6    0.612121
8    0.107659
dtype: float64

With a DataFrame:

In [27]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list(range(0, 12, 2)),
                   columns=list(range(0, 8, 2)))

In [28]:
df1

,0,2,4,6
0,-1.653440,-0.785734,0.014698,0.655930
2,0.650503,-0.122037,1.315481,-0.753851
4,1.188704,-0.783487,-2.154820,0.414759
6,1.662279,-1.113476,-1.882006,0.585102
8,0.041996,0.466142,-0.570213,-0.423184
10,-1.308567,-1.164956,0.912067,-0.283824


Select via integer slicing:

In [29]:
df1.iloc[:3]

,0,2,4,6
0,-1.653440,-0.785734,0.014698,0.655930
2,0.650503,-0.122037,1.315481,-0.753851
4,1.188704,-0.783487,-2.154820,0.414759


In [30]:
df1.iloc[1:5, 2:4]

,4,6
2,1.315481,-0.753851
4,-2.154820,0.414759
6,-1.882006,0.585102
8,-0.570213,-0.423184


Select via integer list:

In [31]:
df1.iloc[[1, 3, 5], [1, 3]]

,2,6
2,-0.122037,-0.753851
6,-1.113476,0.585102
10,-1.164956,-0.283824


In [32]:
df1.iloc[1, 1]

-0.12203673606980107

## Selection by callable

.loc, .iloc, and also [] indexing can accept a callable as indexer. The callable must be a function with one argument (the calling Series or DataFrame) that returns valid output for indexing.

In [33]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))

In [34]:
df1

,A,B,C,D
a,-1.041412,-0.167223,-0.027865,-0.216964
b,0.975096,0.528548,-0.308485,-0.326877
c,-1.480319,1.311950,-1.429285,-0.615063
d,-0.297724,-1.470654,1.599896,0.340529
e,-0.839494,0.517960,0.197426,-0.032205
f,-1.157605,-0.815674,-0.513306,0.890670


In [35]:
df1.loc[lambda df: df.A > 0, :]

,A,B,C,D
b,0.975096,0.528548,-0.308485,-0.326877


In [36]:
df1.loc[:, lambda df: ['A', 'B']]

,A,B
a,-1.041412,-0.167223
b,0.975096,0.528548
c,-1.480319,1.311950
d,-0.297724,-1.470654
e,-0.839494,0.517960
f,-1.157605,-0.815674


In [37]:
df1.iloc[:, lambda df: [0, 1]]

,A,B
a,-1.041412,-0.167223
b,0.975096,0.528548
c,-1.480319,1.311950
d,-0.297724,-1.470654
e,-0.839494,0.517960
f,-1.157605,-0.815674


In [38]:
df1[lambda df: df.columns[0]]

a   -1.041412
b    0.975096
c   -1.480319
d   -0.297724
e   -0.839494
f   -1.157605
Name: A, dtype: float64

You can use callable indexing in Series.

In [39]:
df1.A.loc[lambda s: s > 0]

b    0.975096
Name: A, dtype: float64

## Selecting random samples

In [40]:
s = pd.Series([0, 1, 2, 3, 4, 5])
# When no arguments are passed, returns 1 row.
s.sample()

0    0
dtype: int64

In [41]:
# One may specify either a number of rows:
s.sample(n=3)

3    3
2    2
5    5
dtype: int64

In [42]:
# Or a fraction of the rows:
s.sample(frac=0.5)

4    4
1    1
2    2
dtype: int64

By default, sample will return each row at most once, but one can also sample with replacement using the replace option:

In [43]:
# With replacement:
s.sample(n=6, replace=True)

3    3
2    2
3    3
1    1
4    4
4    4
dtype: int64

## Setting with enlargement

The .loc/[] operations can perform enlargement when setting a non-existent key for that axis.

In the Series case this is effectively an appending operation.

In [44]:
se = pd.Series([1, 2, 3])
se

0    1
1    2
2    3
dtype: int64

In [45]:
se[5] = 5.
se

0    1.0
1    2.0
2    3.0
5    5.0
dtype: float64

A DataFrame can be enlarged on either axis via .loc.

In [46]:
dfi = pd.DataFrame(np.arange(6).reshape(3, 2),
                  columns=['A', 'B'])
dfi

,A,B
0,0,1
1,2,3
2,4,5


In [47]:
dfi.loc[:, 'C'] = dfi.loc[:, 'A']
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4


This is like an append operation on the DataFrame.

In [48]:
dfi.loc[3] = 5
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4
3,5,5,5


## Boolean indexing

Another common operation is the use of boolean vectors to filter the data. The operators are: | for or, & for and, and ~ for not. These must be grouped by using parentheses, since by default Python will evaluate an expression such as df.A > 2 & df.B < 3 as df.A > (2 & df.B) < 3, while the desired evaluation order is (df.A > 2) & (df.B < 3).

Using a boolean vector to index a Series works exactly as in a NumPy ndarray:

In [49]:
s = pd.Series(range(-3, 4))
s

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [50]:
s[s > 0]

4    1
5    2
6    3
dtype: int64

In [51]:
s[(s < -1) | (s > 0.5)]

0   -3
1   -2
4    1
5    2
6    3
dtype: int64

In [52]:
s[~(s < 0)]

3    0
4    1
5    2
6    3
dtype: int64

You may select rows from a DataFrame using a boolean vector the same length as the DataFrame’s index (for example, something derived from one of the columns of the DataFrame):

In [53]:
df[df['A'] > 0]

,A,B,C,D
2000-01-01,0.542045,0.971047,-0.495519,0.137423
2000-01-07,0.345925,-0.122817,0.797087,-3.073061


List comprehensions and the map method of Series can also be used to produce more complex criteria:

In [54]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                    'c': np.random.randn(7)})
df2

,a,b,c
0,one,x,-1.252611
1,one,y,-0.645279
2,two,y,2.426394
3,three,x,0.756417
4,two,y,-0.506433
5,one,x,-0.726977
6,six,x,0.655565


In [55]:
# only want 'two' or 'three'
criterion = df2['a'].map(lambda x: x.startswith('t'))
criterion

0    False
1    False
2     True
3     True
4     True
5    False
6    False
Name: a, dtype: bool

In [56]:
df2[criterion]

,a,b,c
2,two,y,2.426394
3,three,x,0.756417
4,two,y,-0.506433


In [57]:
# equivalent but slower
df2[[x.startswith('t') for x in df2['a']]]

,a,b,c
2,two,y,2.426394
3,three,x,0.756417
4,two,y,-0.506433


In [58]:
# Multiple criteria
df2[criterion & (df2['b'] == 'x')]

,a,b,c
3,three,x,0.756417


With the choice methods Selection by Label, Selection by Position, and Advanced Indexing you may select along more than one axis using boolean vectors combined with other indexing expressions.

In [59]:
df2.loc[criterion & (df2['b'] == 'x'), 'b':'c']

,b,c
3,x,0.756417


## Indexing with isin

Consider the isin() method of Series, which returns a boolean vector that is true wherever the Series elements exist in the passed list. This allows you to select rows where one or more columns have values you want:

In [60]:
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [61]:
s.isin([2, 4, 6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [62]:
s[s.isin([2, 4, 6])]

2    2
0    4
dtype: int64

The same method is available for Index objects and is useful for the cases when you don’t know which of the sought labels are in fact present:

In [63]:
s[s.index.isin([2, 4, 6])]

4    0
2    2
dtype: int64

In [64]:
# compare it to the following
s.reindex([2, 4, 6])

2    2.0
4    0.0
6    NaN
dtype: float64

In addition to that, MultiIndex allows selecting a separate level to use in the membership check:

In [65]:
s_mi = pd.Series(np.arange(6),
                  index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s_mi

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int32

In [66]:
s_mi.iloc[s_mi.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]

0  c    2
1  a    3
dtype: int32

In [67]:
s_mi.iloc[s_mi.index.isin(['a', 'c', 'e'], level=1)]

0  a    0
   c    2
1  a    3
   c    5
dtype: int32

DataFrame also has an isin() method. When calling isin, pass a set of values as either an array or dict. If values is an array, isin returns a DataFrame of booleans that is the same shape as the original DataFrame, with True wherever the element is in the sequence of values.

In [68]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                   'ids2': ['a', 'n', 'c', 'n']})
df

,vals,ids,ids2
0,1,a,a
1,2,b,n
2,3,f,c
3,4,n,n


In [69]:
values = ['a', 'b', 1, 3]

In [70]:
df.isin(values)

,vals,ids,ids2
0,True,True,True
1,False,True,False
2,True,False,False
3,False,False,False


Oftentimes you’ll want to match certain values with certain columns. Just make values a dict where the key is the column, and the value is a list of items you want to check for.

In [71]:
values = {'ids': ['a', 'b'], 'vals': [1, 3]}

In [72]:
df.isin(values)

,vals,ids,ids2
0,True,True,False
1,False,True,False
2,True,False,False
3,False,False,False


Combine DataFrame’s isin with the any() and all() methods to quickly select subsets of your data that meet a given criteria. To select a row where each column meets its own criterion:

In [73]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}

In [74]:
row_mask = df.isin(values).all(1)
df[row_mask]

,vals,ids,ids2
0,1,a,a


## The where() Method and Masking

Selecting values from a Series with a boolean vector generally returns a subset of the data. To guarantee that selection output has the same shape as the original data, you can use the where method in Series and DataFrame.

To return only the selected rows:

In [75]:
s[s > 0]

3    1
2    2
1    3
0    4
dtype: int64

To return a Series of the same shape as the original:

In [76]:
s.where(s > 0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

Selecting values from a DataFrame with a boolean criterion now also preserves input data shape. where is used under the hood as the implementation. The code below is equivalent to df.where(df < 0).

In [77]:
df = pd.DataFrame(np.random.randn(8, 4), index=dates, columns=['A', 'B', 'C', 'D'])

In [78]:
df[df < 0]

,A,B,C,D
2000-01-01,-0.909418,NaN,NaN,-0.306566
2000-01-02,NaN,-0.169101,-0.790862,-0.102295
2000-01-03,-1.297461,-0.656223,NaN,NaN
2000-01-04,NaN,-0.086942,NaN,-0.461271
2000-01-05,NaN,NaN,NaN,NaN
2000-01-06,NaN,-0.314352,NaN,NaN
2000-01-07,NaN,NaN,-2.738897,-0.676540
2000-01-08,NaN,NaN,NaN,NaN


In addition, where takes an optional other argument for replacement of values where the condition is False, in the returned copy.

In [79]:
df.where(df < 0, -df)

,A,B,C,D
2000-01-01,-0.909418,-1.038693,-0.062330,-0.306566
2000-01-02,-0.487326,-0.169101,-0.790862,-0.102295
2000-01-03,-1.297461,-0.656223,-0.631014,-0.793230
2000-01-04,-0.301923,-0.086942,-0.333109,-0.461271
2000-01-05,-0.578371,-0.180206,-0.907961,-0.597696
2000-01-06,-0.405113,-0.314352,-0.658482,-1.002064
2000-01-07,-0.571720,-0.736449,-2.738897,-0.676540
2000-01-08,-1.114886,-0.259167,-0.943092,-1.543070


You may wish to set values based on some boolean criteria. This can be done intuitively like so:

In [80]:
s2 = s.copy()
s2[s2 < 0] = 0
s2

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [81]:
df2 = df.copy()
df2[df2 < 0] = 0
df2

,A,B,C,D
2000-01-01,0.000000,1.038693,0.062330,0.000000
2000-01-02,0.487326,0.000000,0.000000,0.000000
2000-01-03,0.000000,0.000000,0.631014,0.793230
2000-01-04,0.301923,0.000000,0.333109,0.000000
2000-01-05,0.578371,0.180206,0.907961,0.597696
2000-01-06,0.405113,0.000000,0.658482,1.002064
2000-01-07,0.571720,0.736449,0.000000,0.000000
2000-01-08,1.114886,0.259167,0.943092,1.543070


By default, where returns a modified copy of the data. There is an optional parameter inplace so that the original data can be modified without creating a copy:

In [82]:
df_orig = df.copy()
df_orig.where(df > 0, -df, inplace=True)
df_orig

,A,B,C,D
2000-01-01,0.909418,1.038693,0.062330,0.306566
2000-01-02,0.487326,0.169101,0.790862,0.102295
2000-01-03,1.297461,0.656223,0.631014,0.793230
2000-01-04,0.301923,0.086942,0.333109,0.461271
2000-01-05,0.578371,0.180206,0.907961,0.597696
2000-01-06,0.405113,0.314352,0.658482,1.002064
2000-01-07,0.571720,0.736449,2.738897,0.676540
2000-01-08,1.114886,0.259167,0.943092,1.543070


### Alignment

Furthermore, where aligns the input boolean condition (ndarray or DataFrame), such that partial selection with setting is possible. This is analogous to partial setting via .loc (but on the contents rather than the axis labels).

In [83]:
df2 = df.copy()
df2[df2[1:4] > 0] = 3
df2

,A,B,C,D
2000-01-01,-0.909418,1.038693,0.062330,-0.306566
2000-01-02,3.000000,-0.169101,-0.790862,-0.102295
2000-01-03,-1.297461,-0.656223,3.000000,3.000000
2000-01-04,3.000000,-0.086942,3.000000,-0.461271
2000-01-05,0.578371,0.180206,0.907961,0.597696
2000-01-06,0.405113,-0.314352,0.658482,1.002064
2000-01-07,0.571720,0.736449,-2.738897,-0.676540
2000-01-08,1.114886,0.259167,0.943092,1.543070


Where can also accept axis and level parameters to align the input when performing the where.

In [84]:
df2 = df.copy()
df2.where(df2 > 0, df2['A'], axis='index')

,A,B,C,D
2000-01-01,-0.909418,1.038693,0.062330,-0.909418
2000-01-02,0.487326,0.487326,0.487326,0.487326
2000-01-03,-1.297461,-1.297461,0.631014,0.793230
2000-01-04,0.301923,0.301923,0.333109,0.301923
2000-01-05,0.578371,0.180206,0.907961,0.597696
2000-01-06,0.405113,0.405113,0.658482,1.002064
2000-01-07,0.571720,0.736449,0.571720,0.571720
2000-01-08,1.114886,0.259167,0.943092,1.543070


## The query() Method

DataFrame objects have a query() method that allows selection using an expression.

You can get the value of the frame where column b has values between the values of columns a and c. For example:

In [85]:
n = 10

In [86]:
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df

,a,b,c
0,0.908424,0.388245,0.404597
1,0.555787,0.034677,0.881989
2,0.999417,0.875911,0.697915
3,0.412039,0.875874,0.415148
4,0.304421,0.563916,0.898697
5,0.899240,0.012951,0.645819
6,0.583757,0.516434,0.382090
7,0.928993,0.001691,0.148981
8,0.339668,0.869557,0.553435
9,0.879368,0.109711,0.128623


In [87]:
# pure python
df[(df.a < df.b) & (df.b < df.c)]

,a,b,c
4,0.304421,0.563916,0.898697


In [88]:
# query
df.query('(a < b) & (b < c)')

,a,b,c
4,0.304421,0.563916,0.898697


Do the same thing but fall back on a named index if there is no column with the name a.

In [89]:
df = pd.DataFrame(np.random.randint(n / 2, size=(n, 2)), columns=list('bc'))
df.index.name = 'a'
df

,b,c
a,,
0,1,1
1,2,0
2,3,0
3,2,3
4,4,3
5,1,3
6,3,4
7,0,1
8,4,1


In [90]:
df.query('a < b and b < c')

,b,c
a,,


If instead you don’t want to or cannot name your index, you can use the name index in your query expression:

In [91]:
df = pd.DataFrame(np.random.randint(n, size=(n, 2)), columns=list('bc'))
df

,b,c
0,1,4
1,8,8
2,8,9
3,6,0
4,4,7
5,3,7
6,1,1
7,7,4
8,8,4
9,9,5


In [92]:
df.query('index < b < c')

,b,c
0,1,4
2,8,9


## MultiIndex query() Syntax

You can also use the levels of a DataFrame with a MultiIndex as if they were columns in the frame:

In [93]:
n = 10

In [94]:
colors = np.random.choice(['red', 'green'], size=n)
colors

array(['red', 'red', 'red', 'red', 'red', 'red', 'green', 'green',
       'green', 'green'], dtype='<U5')

In [95]:
foods = np.random.choice(['eggs', 'ham'], size=n)
foods

array(['ham', 'ham', 'eggs', 'eggs', 'eggs', 'eggs', 'eggs', 'eggs',
       'eggs', 'eggs'], dtype='<U4')

In [96]:
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])

In [97]:
df = pd.DataFrame(np.random.randn(n, 2), index=index)

In [98]:
df

0         1
color food                    
red   ham   0.341503 -0.697112
      ham   1.143331  0.493137
      eggs  0.763090 -1.584505
      eggs  0.521574  0.376577
      eggs -1.099881  0.446010
      eggs  0.354173  0.819636
green eggs -2.256711  0.542087
      eggs -0.037731  0.352857
      eggs -0.963752  1.390826
      eggs -0.877505  0.255727

In [99]:
df.query('color == "red"')

0         1
color food                    
red   ham   0.341503 -0.697112
      ham   1.143331  0.493137
      eggs  0.763090 -1.584505
      eggs  0.521574  0.376577
      eggs -1.099881  0.446010
      eggs  0.354173  0.819636

If the levels of the MultiIndex are unnamed, you can refer to them using special names:

In [100]:
df.index.names = [None, None]
df

0         1
red   ham   0.341503 -0.697112
      ham   1.143331  0.493137
      eggs  0.763090 -1.584505
      eggs  0.521574  0.376577
      eggs -1.099881  0.446010
      eggs  0.354173  0.819636
green eggs -2.256711  0.542087
      eggs -0.037731  0.352857
      eggs -0.963752  1.390826
      eggs -0.877505  0.255727

In [101]:
df.query('ilevel_0 == "red"')

0         1
red ham   0.341503 -0.697112
    ham   1.143331  0.493137
    eggs  0.763090 -1.584505
    eggs  0.521574  0.376577
    eggs -1.099881  0.446010
    eggs  0.354173  0.819636

#### query() Use Cases

A use case for query() is when you have a collection of DataFrame objects that have a subset of column names (or index levels/names) in common. You can pass the same query to both frames without having to specify which frame you’re interested in querying

In [102]:
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df

,a,b,c
0,0.814189,0.833274,0.427600
1,0.083828,0.350698,0.550696
2,0.187878,0.580243,0.309829
3,0.112394,0.707579,0.593830
4,0.459885,0.477544,0.742091
5,0.858728,0.346716,0.512817
6,0.770165,0.536469,0.492052
7,0.153694,0.249172,0.493870
8,0.838038,0.584314,0.255168
9,0.609922,0.638270,0.198955


In [103]:
df2 = pd.DataFrame(np.random.rand(n + 2, 3), columns=df.columns)
df2

,a,b,c
0,0.833263,0.758818,0.793420
1,0.988505,0.998416,0.014811
2,0.734373,0.933668,0.514030
3,0.987510,0.823450,0.085583
4,0.180899,0.744538,0.469165
5,0.390772,0.190175,0.764046
6,0.344588,0.146359,0.611258
7,0.628458,0.921038,0.023861
8,0.589165,0.038185,0.271746
9,0.891400,0.963155,0.178853


In [104]:
expr = '0.0 <= a <= c <= 0.5'

In [105]:
map(lambda frame: frame.query(expr), [df, df2])

### The in and not in operators

query() also supports special use of Python’s in and not in comparison operators, providing a succinct syntax for calling the isin method of a Series or DataFrame.

In [106]:
# get all rows where columns "a" and "b" have overlapping values
df = pd.DataFrame({'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
                             'c': np.random.randint(5, size=12),
                             'd': np.random.randint(9, size=12)})
df

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3
6,d,b,1,8
7,d,b,4,4
8,e,c,3,2
9,e,c,4,1


In [107]:
df.query('a in b')

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3


In [108]:
df[df.a.isin(df.b)]

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3


In [109]:
df.query('a not in b')

,a,b,c,d
6,d,b,1,8
7,d,b,4,4
8,e,c,3,2
9,e,c,4,1
10,f,c,4,2
11,f,c,0,3


You can combine this with other expressions for very succinct queries:

In [110]:
# rows where cols a and b have overlapping values
# and col c's values are less than col d's
df.query('a in b and c < d')

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3


In [111]:
# pure Python
df[df.b.isin(df.a) & (df.c < df.d)]

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3
6,d,b,1,8
11,f,c,0,3


### Special use of the == operator with list objects

Comparing a list of values to a column using ==/!= works similarly to in/not in.

In [112]:
df.query('b == ["a", "b", "c"]')

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3
6,d,b,1,8
7,d,b,4,4
8,e,c,3,2
9,e,c,4,1


In [113]:
# pure Python
df[df.b.isin(["a", "b", "c"])]

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
2,b,a,2,4
3,b,a,3,8
4,c,b,0,5
5,c,b,1,3
6,d,b,1,8
7,d,b,4,4
8,e,c,3,2
9,e,c,4,1


In [114]:
df.query('c != [1, 2]')

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
3,b,a,3,8
4,c,b,0,5
7,d,b,4,4
8,e,c,3,2
9,e,c,4,1
10,f,c,4,2
11,f,c,0,3


In [115]:
# using in/not in
df.query('[1, 2] in c')

,a,b,c,d
2,b,a,2,4
5,c,b,1,3
6,d,b,1,8


In [116]:
df.query('[1, 2] not in c')

,a,b,c,d
0,a,a,3,6
1,a,a,0,4
3,b,a,3,8
4,c,b,0,5
7,d,b,4,4
8,e,c,3,2
9,e,c,4,1
10,f,c,4,2
11,f,c,0,3


In [117]:
# pure Python
df[df.c.isin([1, 2])]

,a,b,c,d
2,b,a,2,4
5,c,b,1,3
6,d,b,1,8


### Boolean operators

You can negate boolean expressions with the word not or the ~ operator.

In [118]:
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))

In [119]:
df['bools'] = np.random.rand(len(df)) > 0.5

In [120]:
df.query('~bools')

,a,b,c,bools
0,0.794435,0.963006,0.283197,False
5,0.175726,0.530576,0.949796,False
8,0.524196,0.793120,0.674460,False
9,0.406132,0.784641,0.855185,False


In [121]:
df.query('not bools')

,a,b,c,bools
0,0.794435,0.963006,0.283197,False
5,0.175726,0.530576,0.949796,False
8,0.524196,0.793120,0.674460,False
9,0.406132,0.784641,0.855185,False


In [122]:
df.query('not bools') == df[~df.bools]

,a,b,c,bools
0,True,True,True,True
5,True,True,True,True
8,True,True,True,True
9,True,True,True,True


## Duplicate data

If you want to identify and remove duplicate rows in a DataFrame, there are two methods that will help: duplicated and drop_duplicates. Each takes as an argument the columns to use to identify duplicated rows.

- duplicated returns a boolean vector whose length is the number of rows, and which indicates whether a row is duplicated.
- drop_duplicates removes duplicate rows.
By default, the first observed row of a duplicate set is considered unique, but each method has a keep parameter to specify targets to be kept.

- keep='first' (default): mark / drop duplicates except for the first occurrence.
- keep='last': mark / drop duplicates except for the last occurrence.
- keep=False: mark / drop all duplicates.

In [123]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
                    'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'],
                    'c': np.random.randn(7)})

In [124]:
df2

,a,b,c
0,one,x,-0.450025
1,one,y,1.057947
2,two,x,0.167764
3,two,y,-1.699190
4,two,x,0.059692
5,three,x,0.289795
6,four,x,0.044878


In [125]:
df2.duplicated('a')

0    False
1     True
2    False
3     True
4     True
5    False
6    False
dtype: bool

In [126]:
df2.duplicated('a', keep='last')

0     True
1    False
2     True
3     True
4    False
5    False
6    False
dtype: bool

In [127]:
df2.duplicated('a', keep=False)

0     True
1     True
2     True
3     True
4     True
5    False
6    False
dtype: bool

In [128]:
df2.drop_duplicates('a')

,a,b,c
0,one,x,-0.450025
2,two,x,0.167764
5,three,x,0.289795
6,four,x,0.044878


In [129]:
df2.drop_duplicates('a', keep=False)

,a,b,c
5,three,x,0.289795
6,four,x,0.044878


Also, you can pass a list of columns to identify duplications.

In [130]:
df2.duplicated(['a', 'b'])

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [131]:
df2.drop_duplicates(['a', 'b'])

,a,b,c
0,one,x,-0.450025
1,one,y,1.057947
2,two,x,0.167764
3,two,y,-1.699190
5,three,x,0.289795
6,four,x,0.044878


To drop duplicates by index value, use Index.duplicated then perform slicing. The same set of options are available for the keep parameter.

In [132]:
df3 = pd.DataFrame({'a': np.arange(6),
                    'b': np.random.randn(6)},
                    index=['a', 'a', 'b', 'c', 'b', 'a'])
df3

,a,b
a,0,-1.190302
a,1,-1.579655
b,2,0.903773
c,3,1.316083
b,4,2.124853
a,5,-0.142256


In [133]:
df3.index.duplicated()

array([False,  True, False, False,  True,  True])

In [134]:
df3[~df3.index.duplicated()]

,a,b
a,0,-1.190302
b,2,0.903773
c,3,1.316083


In [135]:
df3[~df3.index.duplicated(keep='last')]

,a,b
c,3,1.316083
b,4,2.124853
a,5,-0.142256


In [136]:
df3[~df3.index.duplicated(keep=False)]

,a,b
c,3,1.316083


### Dictionary-like get() method

In [137]:
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])

In [138]:
s.get('a')  # equivalent to s['a']

1

In [139]:
s.get('x', default=-1)

-1

### The lookup() method

Sometimes you want to extract a set of values given a sequence of row labels and column labels, and the lookup method allows for this and returns a NumPy array. For instance:

In [140]:
dflookup = pd.DataFrame(np.random.rand(20, 4), columns = ['A', 'B', 'C', 'D'])
dflookup

,A,B,C,D
0,0.054165,0.039686,0.463959,0.897753
1,0.987206,0.408886,0.368289,0.467334
2,0.327182,0.288902,0.978694,0.206721
3,0.835074,0.042289,0.397429,0.158334
4,0.472961,0.046405,0.835337,0.871208
5,0.473969,0.021123,0.727689,0.839052
6,0.664039,0.252106,0.786200,0.272490
7,0.874370,0.582257,0.953597,0.386553
8,0.807368,0.288600,0.388318,0.211070
9,0.172337,0.538019,0.915884,0.204608


In [141]:
dflookup.lookup(list(range(0, 10, 2)), ['B', 'C', 'A', 'B', 'D'])

array([0.0396864 , 0.97869447, 0.47296127, 0.25210565, 0.2110704 ])

## Index objects

The pandas Index class and its subclasses can be viewed as implementing an ordered multiset. Duplicates are allowed. However, if you try to convert an Index object with duplicate entries into a set, an exception will be raised.

Index also provides the infrastructure necessary for lookups, data alignment, and reindexing. The easiest way to create an Index directly is to pass a list or other sequence to Index:

In [142]:
index = pd.Index(['e', 'd', 'a', 'b'])
index

Index(['e', 'd', 'a', 'b'], dtype='object')

In [143]:
'd' in index

True

In [144]:
index = pd.Index(['e', 'd', 'a', 'b'], name='something')
index.name

'something'

### Setting metadata

Indexes are “mostly immutable”, but it is possible to set and change their metadata, like the index name (or, for MultiIndex, levels and codes).

You can use the rename, set_names, set_levels, and set_codes to set these attributes directly. They default to returning a copy; however, you can specify inplace=True to have the data change in place.

See Advanced Indexing for usage of MultiIndexes.

In [145]:
ind = pd.Index([1, 2, 3])

In [146]:
ind.rename("apple")

Int64Index([1, 2, 3], dtype='int64', name='apple')

In [147]:
ind.set_names(["apple"], inplace=True)

In [148]:
ind.name = "bob"

In [149]:
ind

Int64Index([1, 2, 3], dtype='int64', name='bob')

set_names, set_levels, and set_codes also take an optional level argument

In [150]:
index = pd.MultiIndex.from_product([range(3), ['one', 'two']], names=['first', 'second'])
index

MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two'),
            (2, 'one'),
            (2, 'two')],
           names=['first', 'second'])

In [151]:
index.levels[1]

Index(['one', 'two'], dtype='object', name='second')

In [152]:
index.set_levels(["a", "b"], level=1)

MultiIndex([(0, 'a'),
            (0, 'b'),
            (1, 'a'),
            (1, 'b'),
            (2, 'a'),
            (2, 'b')],
           names=['first', 'second'])

### Set operations on Index objects

The two main operations are union (|) and intersection (&). These can be directly called as instance methods or used via overloaded operators. Difference is provided via the .difference() method.

In [153]:
a = pd.Index(['c', 'b', 'a'])
b = pd.Index(['c', 'e', 'd'])
a | b

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [154]:
a & b

Index(['c'], dtype='object')

In [155]:
a.difference(b)

Index(['a', 'b'], dtype='object')

Also available is the symmetric_difference (^) operation, which returns elements that appear in either idx1 or idx2, but not in both. This is equivalent to the Index created by idx1.difference(idx2).union(idx2.difference(idx1)), with duplicates dropped.

In [156]:
idx1 = pd.Index([1, 2, 3, 4])
idx2 = pd.Index([2, 3, 4, 5])
idx1.symmetric_difference(idx2)

Int64Index([1, 5], dtype='int64')

In [157]:
idx1 ^ idx2

Int64Index([1, 5], dtype='int64')

### Missing values

Index.fillna fills missing values with specified scalar value.

In [158]:
idx1 = pd.Index([1, np.nan, 3, 4])
idx1

Float64Index([1.0, nan, 3.0, 4.0], dtype='float64')

In [159]:
idx1.fillna(2)

Float64Index([1.0, 2.0, 3.0, 4.0], dtype='float64')